### 참고
- https://debuggercafe.com/transfer-learning-with-pytorch/

### 유명한 사전학습 모델 VGG를 사용


In [ ]:
import torch
import torchvision
import torchvision.transforms as transforms
import torch.optim as optim
import time 
import torch.nn.functional as F
import torch.nn as nn
import matplotlib.pyplot as plt
 
from torchvision import models

In [ ]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.manual_seed(777)
if device == 'cuda':
    torch.cuda.manual_seed_all(777)


- VGG는 224*224 입력데이터를 가정하기 때문에, 입력데이터 크기를 맞춰주어야 한다.
Resize((224, 224))

In [ ]:
# Resize(224, 224)

transform = transforms.Compose(
    [transforms.Resize((224, 224)),
     transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
 
trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=32,
                                          shuffle=True)
 
testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=32,
                                         shuffle=False)


Files already downloaded and verified
Files already downloaded and verified


In [ ]:
# 미리학습된 모델 import해서 사용

vgg16 = models.vgg16(pretrained=True)
vgg16.to(device)

print(vgg16)

Downloading: "https://download.pytorch.org/models/vgg16-397923af.pth" to /root/.cache/torch/checkpoints/vgg16-397923af.pth



VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=

## 출력 수 설정
- vgg16.classifier[6].out_features = 10
10개의 클래스로 변경한다.(CIFAR10은 10개의 클래스를 가지고 있다.)
- VGG16을 1000개의 클래스를 주는 이미지넷으로 학습하기때문에 Output은 1000개의 클래스를 맞추도록 설계되어있다. 이를 데이터셋에 맞게 변경시킨다.

## 사전학습 모델 사용시 주의 점
- pretrained model은 이미 잘 학습되어있기 때문에 로드한 가중치를 학습시키지 않고, convolution weight를 freezing 해야한다. VGG16 모델 중 FC이전까지 파라미터 고정시키면 됨

 
- for param in vgg16.features.parameters():

    param.requires_grad = False


In [ ]:
# change the number of classes 
vgg16.classifier[6].out_features = 10
 
# freeze convolution weights
for param in vgg16.features.parameters():
    param.requires_grad = False

In [ ]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(vgg16.classifier.parameters(), lr=0.001, momentum=0.9)


- validate, train 모델함수 정의




In [ ]:
#validation function을 도입

def validate(model, test_dataloader):
    model.eval()
    val_running_loss = 0.0
    val_running_correct = 0
    
    for int, data in enumerate(test_dataloader):
        data, target = data[0].to(device), data[1].to(device)

        output = model(data)
        loss = criterion(output, target)
        
        val_running_loss += loss.item()   # log를 찍기 위해 loss log, accuracy log
        _, preds = torch.max(output.data, 1)
        val_running_correct += (preds == target).sum().item()
    
    val_loss = val_running_loss/len(test_dataloader.dataset)
    val_accuracy = 100. * val_running_correct/len(test_dataloader.dataset)
    
    return val_loss, val_accuracy

In [ ]:
# training function
def train(model, train_dataloader):
    model.train()
    train_running_loss = 0.0
    train_running_correct = 0
    for i, data in enumerate(train_dataloader):
        data, target = data[0].to(device), data[1].to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = criterion(output, target)
        train_running_loss += loss.item()
        _, preds = torch.max(output.data, 1)
        train_running_correct += (preds == target).sum().item()
        loss.backward()
        optimizer.step()
    train_loss = train_running_loss/len(train_dataloader.dataset)
    train_accuracy = 100. * train_running_correct/len(train_dataloader.dataset)
 
    print(f'Train Loss: {train_loss:.4f}, Train Acc: {train_accuracy:.2f}')
    
    return train_loss, train_accuracy

- validate와 train 모델함수를 이용한 모델학습

In [ ]:
train_loss , train_accuracy = [], []
val_loss , val_accuracy = [], []
start = time.time()
for epoch in range(10):
    train_epoch_loss, train_epoch_accuracy = train(vgg16, trainloader)  #train과 validation을 동시에 하고 있다.
    val_epoch_loss, val_epoch_accuracy = validate(vgg16, testloader)
    train_loss.append(train_epoch_loss)
    train_accuracy.append(train_epoch_accuracy)
    val_loss.append(val_epoch_loss)
    val_accuracy.append(val_epoch_accuracy)

    print(train_accuracy, val_accuracy)
    
end = time.time()
 
print((end-start)/60, 'minutes')


# test셋은 정답을 가지고 있지 않으므로 validation을 사용해서 학습모델의 성능을 대략적으로 판단한다.
#[왼쪽은 train의 정확도] [오른쪽은 validation의 정확도]


Train Loss: 0.0257, Train Acc: 73.40
[73.404] [82.63]
Train Loss: 0.0148, Train Acc: 83.37
[73.404, 83.368] [82.63, 84.93]
Train Loss: 0.0118, Train Acc: 86.76
[73.404, 83.368, 86.76] [82.63, 84.93, 86.34]
Train Loss: 0.0094, Train Acc: 89.45
[73.404, 83.368, 86.76, 89.446] [82.63, 84.93, 86.34, 86.59]
Train Loss: 0.0074, Train Acc: 91.78
[73.404, 83.368, 86.76, 89.446, 91.778] [82.63, 84.93, 86.34, 86.59, 86.91]
Train Loss: 0.0057, Train Acc: 93.63
[73.404, 83.368, 86.76, 89.446, 91.778, 93.63] [82.63, 84.93, 86.34, 86.59, 86.91, 87.01]
Train Loss: 0.0043, Train Acc: 95.23
[73.404, 83.368, 86.76, 89.446, 91.778, 93.63, 95.226] [82.63, 84.93, 86.34, 86.59, 86.91, 87.01, 87.31]
Train Loss: 0.0031, Train Acc: 96.80


In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_accuracy, color='green', label='train accuracy')
plt.plot(val_accuracy, color='blue', label='validataion accuracy')
plt.legend()
plt.savefig('accuracy.png')
plt.show()

In [ ]:
plt.figure(figsize=(10, 7))
plt.plot(train_loss, color='orange', label='train loss')
plt.plot(val_loss, color='red', label='validataion loss')
plt.legend()
plt.savefig('loss.png')
plt.show()

